# 단어 임베딩(단어 수치화 전처리)

## one-hot -encoding

In [4]:
#원-핫 인코딩
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = '오늘 날씨는 구름이 많아요'

#명사만 추출
nouns = komoran.nouns(text)
print(nouns)

#단어 사전 구축 및 단어별 인덱스 부여
dics ={}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)   

# 원-핫 인코딩
nb_classes = len(dics) #원-핫 벡터 차원의 크기를 결정, 단어 사전의 크기가 원핫 벡터의 크기
targets = list(dics.values()) #원핫 인코딩 기능을 사용하기 위해 딕셔너리의 value값을 list로 변환
one_hot_targets = np.eye(nb_classes)[targets]
# eye함수는 단위행렬을 만들어줌 eye함수 뒤에 붙은 [target]를 이용해 생성된 단위행렬의 순서를 단어 사전의 순서에 맞게 정렬해줌
print(one_hot_targets)

#오늘: 1 0 0
#날씨: 0 1 0
#구름: 0 0 1

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Word2Vec

In [5]:
#에제로 네이버 영화 리뷰 말뭉치 데이터 사용
#자연어 처리 위한 라이브러리 Gensim패키지 사용
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

In [13]:
# 네이버 영화 리뷰 데이터 읽어오기
#한글파일(웹페이지)는 왠만하면 utf-8 아니면 euc-kr로 인코딩 되어있다!
def read_review_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] #헤더 제거
    return data

#들어온 데이터가 /t를 기준으로 데이터를 분리함
#그 후 첫 번째 행의 헤더를 제거하고 리뷰 데이터만 반환함

In [14]:
#학습 시간 측정 시작
start = time.time()

#리뷰 파일 읽어오기
print('말뭉치 데이터 읽기 시작')
review_data = read_review_data('./nsmc-master/ratings.txt') 
# read_review_data 함수를 호출해 ratings.txt 파일을 리스트 형태로 읽어옴
print(len(review_data)) # 리뷰 데이터 전체 개수
print('말뭉치 데이터 읽기 완료: ', time.time() - start)

말뭉치 데이터 읽기 시작
200000
말뭉치 데이터 읽기 완료:  1.8244166374206543


In [15]:
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data] 
# sentence[1]은 rating.txt파일에서 document 칼럼의 데이터를 의미함
print('형태소에서 명사만 추출완료: ', time.time() - start)

형태소에서 명사만 추출 시작
형태소에서 명사만 추출완료:  287.51341009140015


In [17]:
# word2vec 모델 학습
print('word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print('word2vec 모델 학습 완료: ', time.time() - start)
#명사만 추출한 곳에서 word2vec 모델을 학습시킴 
#주요 하이퍼파라미터
# sentences: word2vec 모델 학습에 필요한 문장 데이터. word2vec 모델의 입력값으로 사용됨
# size: 단어 임베딩 벡터의 차원(크기)
# winodw: 주변 단어 윈도우의 크기
# hs: 0(0이 아닌 경우 음수 샘플링 사용), 1 (모델 학습에 softmax사용)
# min_count: 단어 최소 빈도 수 제한(설정된 빈도 수 이하의 단어들은 학습하지 않음)
#sg: 0(cbow 모델) 1(skip-gram 모델)

word2vec 모델 학습 시작
word2vec 모델 학습 완료:  361.64774775505066


In [18]:
# 모델 저장
print('모델 저장 시작')
model.save('nvmc.model')
print('학습된 모델 저장 완료: ',time.time()-start)

모델 저장 시작
학습된 모델 저장 완료:  400.1078898906708


In [19]:
#학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print('corpus_count:', model.corpus_count)
print('corpus_total_words: ', model.corpus_total_words)

corpus_count: 200000
corpus_total_words:  1076896


### 위에서 한 예제 유사도 확인해보기

In [21]:
#위에서 생성된 Word2Vec 모델 파일을 읽어와 실제 단어 임베딩된 값과 벡터 공간상의 유사한 단어들 확인 예제
from gensim.models import Word2Vec

#모델 로딩
model = Word2Vec.load('nvmc.model')
print('coupu_total_words: ', model.corpus_total_words)

#'사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑: ',model.wv['사랑'])
# 사랑 이라는 단어로 생성한 단어 임베딩 벡터이다.
#모델을 학습할 때 설정한 size 하이퍼파라미터만큼 단어 임베딩 벡터 차원 크기가 결정됨

coupu_total_words:  1076896
사랑:  [-4.15605679e-02 -3.80127549e-01  1.61127716e-01  5.71583118e-03
 -5.89701124e-02 -3.86713356e-01 -4.48233373e-02  1.22257210e-01
  1.86522473e-02 -1.58316374e-01 -3.80085111e-02  4.05164100e-02
  1.96985215e-01  8.85017291e-02 -2.34512106e-01  3.29363614e-01
 -3.15285288e-03 -2.53304821e-02  1.37879848e-02 -4.56013292e-01
  1.90241456e-01  3.94645557e-02 -2.20273763e-01  3.70881647e-01
 -3.90942931e-01 -1.29743010e-01  1.65335774e-01 -4.33890760e-01
 -1.17295861e-01 -1.13350980e-01  3.92541029e-02  2.02908397e-01
  7.84973148e-03 -2.42442951e-01  1.64653763e-01  4.73551959e-01
  1.20241977e-01 -1.18682496e-01 -2.67837882e-01  9.67743471e-02
 -4.64657135e-03  1.07407257e-01 -9.09824073e-02 -2.58279264e-01
  4.84912902e-01  3.10303032e-01 -3.22651267e-01  1.51932975e-02
  2.50542164e-01 -2.45144013e-02  1.88086197e-01 -2.68738002e-01
 -1.06334642e-01 -3.18943150e-02  1.01010732e-01  2.60324031e-01
  2.89106935e-01 -2.40909040e-01  1.76846668e-01 -2.42138

In [22]:
#단어 유사도 계산
print('일요일 = 월요일\t', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우\t', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성\t', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 = 삼성\t', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 = 월요일\t', model.wv.similarity(w1='히어로', w2='월요일'))

#gensim 패키지의 model.wv.most_similar() 함수를 호출할 경우 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환해줌
# 즉 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환함

일요일 = 월요일	 0.6664964
안성기 = 배우	 0.56402194
대기업 = 삼성	 0.54993635
일요일 = 삼성	 0.2801627
히어로 = 월요일	 0.04280297


In [23]:
#가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5)) #topn 인자는 반환되는 유사한 단어 수를 의미하며 topn=5는 5개까지 유사한 단어를 반환했다.
print(model.wv.most_similar('시리즈', topn=5))

[('씨야', 0.7382012009620667), ('이은우', 0.7302901744842529), ('능청', 0.7158526182174683), ('김수로', 0.7059884071350098), ('김갑수', 0.7055827975273132)]
[('더 울버린', 0.672785758972168), ('X맨', 0.6529032588005066), ('나니아 연대기', 0.6478983163833618), ('기사단', 0.647780179977417), ('파라노말 액티비티', 0.6380230188369751)]
